In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
#dd-ignore

!pip install --user dd-scenario


In [0]:
#dd-ignore

from dd_scenario import *

# Creates a client...
# If you want to be able to call solve() on the client, you have to provide your API Key
# client = Client(pc=pc, apikey='IAM_APIKEY')
client = Client(pc=pc)


In [0]:
#dd-ignore

#Get 'Loan Department' decision...
dd_model_builder = client.get_model_builder(name="Loan Department")

#Get scenario 'ScheduleActivities'...
scenario = dd_model_builder.get_scenario(name="ScheduleActivities")

#Load all input data as a map { data_name: data_frame }
inputs = scenario.get_tables_data(category='input')
# This will hold all outputs as a map { data_name: data_frame }
outputs = {}

# we use a lock to access ``outputs``. This allows solves() to
# be aborted without race condition in data writting
import threading
output_lock = threading.Lock()



In [0]:
from docplex.mp.utils import *
from docplex.cp.model import *
from docplex.cp.expression import _FLOATING_POINT_PRECISION
from docplex.util.environment import get_environment
import time
import operator

import pandas as pd
import numpy as np
import math

import codecs
import sys

# Handle output of unicode strings
if sys.version_info[0] < 3:
    sys.stdout = codecs.getwriter('utf8')(sys.stdout)


# Convert type to 'int64'
def helper_int64_convert(arg):
    if pd.__version__ < '0.20.0':
        return arg.astype('int64', raise_on_error=False)
    else:
        return arg.astype('int64', errors='ignore')

# Parse and convert an integer Series to a date Series
# Integer value represents the number of schedule units (time granularity for engine) since horizon start
def helper_convert_int_series_to_date(sched_int_series):
    return pd.to_datetime(sched_int_series * secs_per_day / duration_units_per_day / schedUnitPerDurationUnit * nanosecs_per_sec + horizon_start_date.value, errors='coerce')

# Convert a duration Series to a Series representing the number of scheduling units
def helper_convert_duration_series_to_scheduling_unit(duration_series, nb_input_data_units_per_day):
    return helper_int64_convert(duration_series * duration_units_per_day * schedUnitPerDurationUnit / nb_input_data_units_per_day)

def helper_get_column_name_for_property(property):
    return helper_property_id_to_column_names_map.get(property, 'unknown')


# Parse and convert a date to an integer
# Integer value represents the number of schedule units (time granularity for engine) since horizon start
def helper_convert_date_to_int(date):
    return int((date - horizon_start_date).value / nanosecs_per_sec * duration_units_per_day * schedUnitPerDurationUnit / secs_per_day)


def helper_parse_and_convert_date_to_int(date_as_str):
    return helper_convert_date_to_int(pd.to_datetime(date_as_str))

# Label constraint
expr_counter = 1
def helper_add_labeled_cpo_constraint(mdl, expr, label, context=None, columns=None):
    global expr_counter
    if isinstance(expr, np.bool_):
        expr = expr.item()
    if isinstance(expr, bool):
        pass  # Adding a trivial constraint: if infeasible, docplex will raise an exception it is added to the model
    else:
        expr.name = '_L_EXPR_' + str(expr_counter)
        expr_counter += 1
        if columns:
            ctxt = ", ".join(str(getattr(context, col)) for col in columns)
        else:
            if context:
                ctxt = context.Index if isinstance(context.Index, str) is not None else ", ".join(context.Index)
            else:
                ctxt = None
        expr_to_info[expr.name] = (label, ctxt)
    mdl.add(expr)

def helper_get_index_names_for_type(dataframe, type):
    if not is_pandas_dataframe(dataframe):
        return None
    return [name for name in dataframe.index.names if name in helper_concept_id_to_index_names_map.get(type, [])]


helper_concept_id_to_index_names_map = {
    'cTask': ['id_of_Activity'],
    'LoanOfficers': ['id_of_LoanOfficers'],
    'Activity': ['id_of_Activity'],
    'cUnaryResource': ['id_of_LoanOfficers']}
helper_property_id_to_column_names_map = {
    'Activity.Duration in days': 'Duration_in_days',
    'Activity.Activity': 'Activity',
    'cTask.fixedDuration': 'Duration_in_days',
    'LoanOfficers.Name': 'Name'}


# Data model definition for each table
# Data collection: list_of_Activity ['Duration_in_days', 'Activity']
# Data collection: list_of_LoanOfficers ['Name']

# Create a pandas Dataframe for each data table
list_of_Activity = inputs[u'Activity']
list_of_Activity = list_of_Activity[[u'Duration in days', u'Activity']].copy()
list_of_Activity.rename(columns={u'Duration in days': 'Duration_in_days', u'Activity': 'Activity'}, inplace=True)
list_of_LoanOfficers = inputs[u'LoanOfficers']
list_of_LoanOfficers = list_of_LoanOfficers[[u'Name']].copy()
list_of_LoanOfficers.rename(columns={u'Name': 'Name'}, inplace=True)

# Set index when a primary key is defined
list_of_Activity.set_index('Activity', inplace=True)
list_of_Activity.sort_index(inplace=True)
list_of_Activity.index.name = 'id_of_Activity'
list_of_LoanOfficers.set_index('Name', inplace=True)
list_of_LoanOfficers.sort_index(inplace=True)
list_of_LoanOfficers.index.name = 'id_of_LoanOfficers'
# Define time granularity for scheduling
schedUnitPerDurationUnit = 1440  # DurationUnit is days
duration_units_per_day = 1.0


# Define global constants for date to integer conversions
horizon_start_date = pd.to_datetime('Wed Jul 25 00:00:00 UTC 2018')
horizon_end_date = horizon_start_date + pd.Timedelta(days=3650)
nanosecs_per_sec = 1000.0 * 1000 * 1000
secs_per_day = 3600.0 * 24

# Convert all input durations to internal time unit
list_of_Activity['INTERNAL_RAW_Duration_in_days'] = list_of_Activity['Duration_in_days']
list_of_Activity['Duration_in_days'] = helper_convert_duration_series_to_scheduling_unit(list_of_Activity.Duration_in_days, 1.0)


# Create data frame as cartesian product of: Activity x LoanOfficers
list_of_SchedulingAssignment = pd.DataFrame(index=pd.MultiIndex.from_product((list_of_Activity.index, list_of_LoanOfficers.index), names=['id_of_Activity', 'id_of_LoanOfficers']))




def build_model():
    mdl = CpoModel()

    # Definition of model variables
    list_of_SchedulingAssignment['interval'] = interval_var_list(len(list_of_SchedulingAssignment), end=(INTERVAL_MIN, helper_convert_date_to_int(horizon_end_date)), optional=True)
    list_of_SchedulingAssignment['schedulingAssignmentVar'] = list_of_SchedulingAssignment.interval.apply(mdl.presence_of)
    list_of_Activity['interval'] = interval_var_list(len(list_of_Activity), end=(INTERVAL_MIN, helper_convert_date_to_int(horizon_end_date)), optional=True)
    list_of_Activity['taskStartVar'] = list_of_Activity.interval.apply(mdl.start_of)
    list_of_Activity['taskEndVar'] = list_of_Activity.interval.apply(mdl.end_of)
    list_of_Activity['taskDurationVar'] = list_of_Activity.interval.apply(mdl.size_of)
    list_of_Activity['taskAbsenceVar'] = 1 - list_of_Activity.interval.apply(mdl.presence_of)
    list_of_Activity['taskPresenceVar'] = list_of_Activity.interval.apply(mdl.presence_of)


    # Definition of model
    # Objective cMinimizeMakespan-
    # Combine weighted criteria: 
    # 	cMinimizeMakespan cMinimizeMakespan 1.2{
    # 	task = Activity,
    # 	scaleFactorExpr = 1,
    # 	(static) goalFilter = null,
    # 	(static) taskEnd = decisionPath(cTaskEnd[Activity]),
    # 	(static) numericExpr = max of decisionPath(cTaskEnd[Activity]) over cTaskEnd[Activity]} with weight 5.0
    agg_Activity_taskEndVar_SG1 = mdl.max(list_of_Activity.taskEndVar)
    
    kpis_expression_list = [
        (1, 1.0, agg_Activity_taskEndVar_SG1 / schedUnitPerDurationUnit, 1, 0, u'time to complete all Activities')]
    custom_code.update_goals_list(kpis_expression_list)
    
    for i, (_, kpi_weight, kpi_expr, kpi_factor, kpi_offset, kpi_name) in enumerate(kpis_expression_list):
        kpi_var = integer_var(name='kpi_' + repr(i + 1))
        mdl.add(kpi_var >= kpi_weight * ((kpi_expr * kpi_factor) - kpi_offset) - 1 + _FLOATING_POINT_PRECISION)
        mdl.add(kpi_var <= kpi_weight * ((kpi_expr * kpi_factor) - kpi_offset))
        mdl.add_kpi(kpi_var, name=kpi_name)
    
    mdl.add(minimize(sum([kpi_sign * kpi_weight * ((kpi_expr * kpi_factor) - kpi_offset) for kpi_sign, kpi_weight, kpi_expr, kpi_factor, kpi_offset, kpi_name in kpis_expression_list])))
    
    # [ST_1] Constraint : cLimitNumberOfResourcesAssignedToEachActivitySched_cIterativeRelationalConstraint
    # The number of LoanOfficers assignments for each Activity is equal to 1
    # Label: CT_1_The_number_of_LoanOfficers_assignments_for_each_Activity_is_equal_to_1
    groupbyLevels = ['id_of_Activity']
    groupby_SchedulingAssignment = list_of_SchedulingAssignment.schedulingAssignmentVar.groupby(level=groupbyLevels).sum().to_frame()
    for row in groupby_SchedulingAssignment.itertuples(index=True):
        helper_add_labeled_cpo_constraint(mdl, row.schedulingAssignmentVar == 1, u'The number of LoanOfficers assignments for each Activity is equal to 1', row)
    
    # [ST_2] Constraint : cSetFixedDurationSpezProp_cSetFixedDurationPath
    # The schedule must respect the duration specified for each Activity
    # Label: CT_2_The_schedule_must_respect_the_duration_specified_for_each_Activity
    for row in list_of_Activity[list_of_Activity.Duration_in_days.notnull()].itertuples(index=True):
        helper_add_labeled_cpo_constraint(mdl, size_of(row.interval, int(row.Duration_in_days)) == int(row.Duration_in_days), u'The schedule must respect the duration specified for each Activity', row)
    
    # [ST_3] Constraint : cForceTaskPresence_cIterativeRelationalConstraint
    # All Activities are scheduled
    # Label: CT_3_All_Activities_are_scheduled
    for row in list_of_Activity.itertuples(index=True):
        helper_add_labeled_cpo_constraint(mdl, row.taskAbsenceVar != 1, u'All Activities are scheduled', row)
    
    # Scheduling internal structure
    groupby_SchedulingAssignment = list_of_SchedulingAssignment.reset_index()[['id_of_Activity', 'interval']].groupby(['id_of_Activity'])['interval'].apply(list).to_frame()
    join_SchedulingAssignment = groupby_SchedulingAssignment.join(list_of_Activity.interval, rsuffix='_right', how='inner')
    for row in join_SchedulingAssignment.itertuples(index=False):
        mdl.add(synchronize(row.interval_right, row.interval))
    
    # link presence if not alternative
    groupby_SchedulingAssignment = list_of_SchedulingAssignment.schedulingAssignmentVar.groupby(level=['id_of_Activity']).agg(lambda l: mdl.max(l.tolist())).to_frame()
    join_SchedulingAssignment = groupby_SchedulingAssignment.join(list_of_Activity.taskPresenceVar, how='inner')
    for row in join_SchedulingAssignment.itertuples(index=False):
        mdl.add(row.schedulingAssignmentVar <= row.taskPresenceVar)
    
    # no overlap
    groupby_SchedulingAssignment = list_of_SchedulingAssignment.reset_index()[['id_of_LoanOfficers', 'interval']].groupby(['id_of_LoanOfficers'])['interval'].apply(list).to_frame()
    for row in groupby_SchedulingAssignment.reset_index().itertuples(index=False):
        mdl.add(no_overlap(row.interval))


    return mdl


def solve_model(mdl):
    params = CpoParameters()
    params.TimeLimit = 120
    # Call to custom code to update parameters value
    custom_code.update_solver_params(params)
    # Update parameters value according to environment variables definition
    cpo_param_env_prefix = 'ma.cpo.'
    cpo_params = [name[4:] for name in dir(CpoParameters) if name.startswith('set_')]
    for param in cpo_params:
        env_param = cpo_param_env_prefix + param
        param_value = get_environment().get_parameter(env_param)
        if param_value:
            # Updating parameter value
            print("Updated value for parameter %s = %s" % (param, param_value))
            params[param] = param_value

    solver = CpoSolver(mdl, params=params, trace_log=True)
    try:
        for i, msol in enumerate(solver):
            ovals = msol.get_objective_values()
            print("Objective values: {}".format(ovals))
            # Initialize dict iterator that works in Py2.7 and Py3
            kpis_dict = msol.get_kpis()
            kpis_iterator = getattr(kpis_dict, "viewitems", None)
            if not kpis_iterator:
                kpis_iterator = kpis_dict.items
            for k, v in kpis_iterator():
                print('%s --> %s' % (k, v))
            export_solution(msol)
            if ovals is None:
                break  # No objective: stop after first solution
        # If model is infeasible, invoke conflict refiner to return
        if solver.get_last_solution().get_solve_status() == SOLVE_STATUS_INFEASIBLE:
            conflicts = solver.refine_conflict()
            export_conflicts(conflicts)
    except CpoException as e:
        # Solve has been aborted from an external action
        print('An exception has been raised: %s' % str(e))
        raise e


expr_to_info = {}


def export_conflicts(conflicts):
    # Display conflicts in console
    print('Conflict set:')
    list_of_conflicts = pd.DataFrame(columns=['constraint', 'context', 'detail'])
    for item, index in zip(conflicts.member_constraints, range(len(conflicts.member_constraints))):
        label, context = expr_to_info.get(item.name, ('N/A', item.name))
        constraint_detail = expression._to_string(item)
        # Print conflict information in console
        print("Conflict involving constraint: %s, \tfor: %s -> %s" % (label, context, constraint_detail))
        list_of_conflicts = list_of_conflicts.append(pd.DataFrame({'constraint': label, 'context': str(context), 'detail': constraint_detail},
                                                                  index=[index], columns=['constraint', 'context', 'detail']))

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_conflicts'] = list_of_conflicts


def export_solution(msol):
    start_time = time.time()
    list_of_SchedulingAssignment_solution = pd.DataFrame(index=list_of_SchedulingAssignment.index)
    list_of_SchedulingAssignment_solution['schedulingAssignmentVar'] = list_of_SchedulingAssignment.interval.apply(lambda r: (1 if msol.solution.get_var_solution(r).is_present() else 0) if msol.solution.get_var_solution(r) else np.NaN)
    list_of_Activity_solution = pd.DataFrame(index=list_of_Activity.index)
    list_of_Activity_solution = list_of_Activity_solution.join(pd.DataFrame([msol.solution[interval] if msol.solution[interval] else (None, None, None) for interval in list_of_Activity.interval], index=list_of_Activity.index, columns=['taskStartVar', 'taskEndVar', 'taskDurationVar']))
    list_of_Activity_solution['taskStartVarDate'] = helper_convert_int_series_to_date(list_of_Activity_solution.taskStartVar)
    list_of_Activity_solution['taskEndVarDate'] = helper_convert_int_series_to_date(list_of_Activity_solution.taskEndVar)
    list_of_Activity_solution.taskStartVar /= schedUnitPerDurationUnit
    list_of_Activity_solution.taskEndVar /= schedUnitPerDurationUnit
    list_of_Activity_solution.taskDurationVar /= schedUnitPerDurationUnit
    list_of_Activity_solution['taskAbsenceVar'] = list_of_Activity.interval.apply(lambda r: (1 if msol.solution.get_var_solution(r).is_absent() else 0) if msol.solution.get_var_solution(r) else np.NaN)
    list_of_Activity_solution['taskPresenceVar'] = list_of_Activity.interval.apply(lambda r: (1 if msol.solution.get_var_solution(r).is_present() else 0) if msol.solution.get_var_solution(r) else np.NaN)

    # Filter rows for non-selected assignments
    list_of_SchedulingAssignment_solution = list_of_SchedulingAssignment_solution[list_of_SchedulingAssignment_solution.schedulingAssignmentVar > 0.5]

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_Activity_solution'] = list_of_Activity_solution.reset_index()
        outputs['list_of_SchedulingAssignment_solution'] = list_of_SchedulingAssignment_solution.reset_index()
        custom_code.post_process_solution(msol, outputs)

    elapsed_time = time.time() - start_time
    print('solution export done in ' + str(elapsed_time) + ' secs')
    return


# Instantiate CustomCode class if definition exists
try:
    custom_code = CustomCode(globals())
except NameError:
    # Create a dummy anonymous object for custom_code
    custom_code = type('', (object,), {'preprocess': (lambda *args: None),
                                       'update_goals_list': (lambda *args: None),
                                       'update_model': (lambda *args: None),
                                       'update_solver_params': (lambda *args: None),
                                       'post_process_solution': (lambda *args: None)})()

# Custom pre-process
custom_code.preprocess()

print('* building wado model')
start_time = time.time()
model = build_model()

# Model customization
custom_code.update_model(model)

elapsed_time = time.time() - start_time
print('model building done in ' + str(elapsed_time) + ' secs')

print('* running wado model')
start_time = time.time()
solve_model(model)
elapsed_time = time.time() - start_time
print('solve + export of all intermediate solutions done in ' + str(elapsed_time) + ' secs')